In [1]:
from analysis_classes import PredictiveRegression
import pandas as pd

tsr = PredictiveRegression()
keys = ['year','GDP', 'Fed_Budget', 'inflation_rate', 'PA_Budget_diff', 'CPI_inflation', 'Labor_BS', 'Labor_cond', 'Unemploy_BS', 'Unemploy']
X = pd.read_csv('../../data-collection/data_interpolated.csv')[keys[0]]
Y = pd.read_csv('../../data-collection/data_interpolated.csv')['faculty']

In [2]:
tsr.time_series_regression(X, Y)

{'trend': 57              NaN
 58              NaN
 59              NaN
 60     4.787302e+05
 61     5.032727e+05
 62     5.278151e+05
 63     5.523575e+05
 64     5.768999e+05
 65     6.014423e+05
 66     6.259847e+05
 67     6.505272e+05
 68     6.750696e+05
 69     6.996120e+05
 70     7.241544e+05
 71     7.486968e+05
 72     7.732392e+05
 73     7.977816e+05
 74     8.223241e+05
 75     8.468665e+05
 76     8.714089e+05
 77     8.959513e+05
 78     9.204937e+05
 79     9.450361e+05
 80     9.695786e+05
 81     9.941210e+05
 82     1.018663e+06
 83     1.043206e+06
 84     1.067748e+06
 85     1.092291e+06
 86     1.116833e+06
 87     1.141375e+06
 88     1.165918e+06
 89     1.190460e+06
 90     1.215003e+06
 91     1.239545e+06
 92     1.264088e+06
 93     1.288630e+06
 94     1.313172e+06
 95     1.337715e+06
 96     1.362257e+06
 97     1.386800e+06
 98     1.411342e+06
 99     1.435884e+06
 100    1.460427e+06
 101    1.484969e+06
 102    1.509512e+06
 103    1.534054e+06
 104

In [3]:
tsr.lowess_regression(X,Y)

{'lag': 3,
 'prediction': 1566358.011657067,
 'r2': np.float64(0.9973215435462112),
 'rmse': np.float64(18330.546937859996),
 'mae': np.float64(14482.099965163208),
 'aic': np.float64(1011.632417878794),
 'plot_data':           X     Y_data        Y_pred
 57   1970.0   474000.0  4.690638e+05
 58   1971.0   492000.0  4.939404e+05
 59   1972.0   500000.0  5.185428e+05
 60   1973.0   527000.0  5.427620e+05
 61   1974.0   567000.0  5.665474e+05
 62   1975.0   628000.0  5.899792e+05
 63   1976.0   633000.0  6.134385e+05
 64   1977.0   678000.0  6.378634e+05
 65   1978.0   676500.0  6.579254e+05
 66   1979.0   675000.0  6.750753e+05
 67   1980.0   686000.0  6.882284e+05
 68   1981.0   705000.0  6.980730e+05
 69   1982.0   710000.0  7.079483e+05
 70   1983.0   724000.0  7.201078e+05
 71   1984.0   717000.0  7.343088e+05
 72   1985.0   715000.0  7.490042e+05
 73   1986.0   722000.0  7.635577e+05
 74   1987.0   793070.0  7.788144e+05
 75   1988.0   808645.0  7.960651e+05
 76   1989.0   824220.0

In [4]:
tsr.arima_regression(X,Y)

{'lag': 3,
 'prediction': 1548862.8413066887,
 'r2': np.float64(0.9837409817014481),
 'rmse': np.float64(45162.74060831409),
 'mae': np.float64(20133.455593276736),
 'aic': np.float64(1089.8027703054527),
 'params': (3, 2, 3),
 'plot_data':           X     Y_data        Y_pred
 57   1970.0   474000.0  2.308028e+05
 58   1971.0   492000.0  6.640755e+05
 59   1972.0   500000.0  4.912610e+05
 60   1973.0   527000.0  5.526671e+05
 61   1974.0   567000.0  5.735523e+05
 62   1975.0   628000.0  6.398214e+05
 63   1976.0   633000.0  6.720874e+05
 64   1977.0   678000.0  6.545505e+05
 65   1978.0   676500.0  7.007894e+05
 66   1979.0   675000.0  6.835679e+05
 67   1980.0   686000.0  6.755565e+05
 68   1981.0   705000.0  7.033368e+05
 69   1982.0   710000.0  7.139041e+05
 70   1983.0   724000.0  7.193147e+05
 71   1984.0   717000.0  7.316056e+05
 72   1985.0   715000.0  7.204977e+05
 73   1986.0   722000.0  7.171741e+05
 74   1987.0   793070.0  7.308742e+05
 75   1988.0   808645.0  8.290449e+05


In [5]:
tsr.gaussian_process_regression(X,Y)

{'lag': 3,
 'prediction': 1539651.9183053442,
 'r2': np.float64(0.9988701949606721),
 'rmse': np.float64(11905.15028659528),
 'mae': np.float64(9052.57609426921),
 'aic': np.float64(942.472638266281),
 'std': 89505.45574289898,
 'plot_data':           X     Y_data        Y_pred         Y_std
 57   1970.0   474000.0  4.630435e+05  16962.376733
 58   1971.0   492000.0  4.867165e+05  14925.661163
 59   1972.0   500000.0  5.153157e+05  14536.712378
 60   1973.0   527000.0  5.463997e+05  14531.511562
 61   1974.0   567000.0  5.776442e+05  14479.630751
 62   1975.0   628000.0  6.070484e+05  14367.401702
 63   1976.0   633000.0  6.331011e+05  14270.538783
 64   1977.0   678000.0  6.548882e+05  14225.303484
 65   1978.0   676500.0  6.721306e+05  14217.649122
 66   1979.0   675000.0  6.851505e+05  14216.922783
 67   1980.0   686000.0  6.947727e+05  14204.760888
 68   1981.0   705000.0  7.021742e+05  14181.890577
 69   1982.0   710000.0  7.087045e+05  14159.004142
 70   1983.0   724000.0  7.1569

In [6]:
tsr.linear_regression(X,Y)

{'lag': 3,
 'prediction': np.float64(1722080.664201662),
 'r2': np.float64(0.9674182336397756),
 'rmse': np.float64(63932.316524036716),
 'mae': np.float64(57804.31139495805),
 'aic': np.float64(1110.5580248424744),
 'plot_data':           X     Y_data        Y_pred
 57   1970.0   474000.0  4.184892e+05
 58   1971.0   492000.0  4.426298e+05
 59   1972.0   500000.0  4.667703e+05
 60   1973.0   527000.0  4.909109e+05
 61   1974.0   567000.0  5.150515e+05
 62   1975.0   628000.0  5.391921e+05
 63   1976.0   633000.0  5.633327e+05
 64   1977.0   678000.0  5.874733e+05
 65   1978.0   676500.0  6.116138e+05
 66   1979.0   675000.0  6.357544e+05
 67   1980.0   686000.0  6.598950e+05
 68   1981.0   705000.0  6.840356e+05
 69   1982.0   710000.0  7.081762e+05
 70   1983.0   724000.0  7.323168e+05
 71   1984.0   717000.0  7.564573e+05
 72   1985.0   715000.0  7.805979e+05
 73   1986.0   722000.0  8.047385e+05
 74   1987.0   793070.0  8.288791e+05
 75   1988.0   808645.0  8.530197e+05
 76   1989.

In [7]:
tsr.polynomial_regression(X,Y)

{'lag': 3,
 'prediction': np.float64(1860268.749033451),
 'r2': np.float64(0.9794546525397061),
 'rmse': np.float64(50767.97882364732),
 'mae': np.float64(41980.10976333686),
 'aic': np.float64(1089.5021096702799),
 'plot_data':           X     Y_data        Y_pred
 57   1970.0   474000.0  5.003166e+05
 58   1971.0   492000.0  5.144375e+05
 59   1972.0   500000.0  5.289759e+05
 60   1973.0   527000.0  5.439317e+05
 61   1974.0   567000.0  5.593051e+05
 62   1975.0   628000.0  5.750959e+05
 63   1976.0   633000.0  5.913043e+05
 64   1977.0   678000.0  6.079301e+05
 65   1978.0   676500.0  6.249734e+05
 66   1979.0   675000.0  6.424342e+05
 67   1980.0   686000.0  6.603125e+05
 68   1981.0   705000.0  6.786083e+05
 69   1982.0   710000.0  6.973215e+05
 70   1983.0   724000.0  7.164523e+05
 71   1984.0   717000.0  7.360005e+05
 72   1985.0   715000.0  7.559662e+05
 73   1986.0   722000.0  7.763494e+05
 74   1987.0   793070.0  7.971501e+05
 75   1988.0   808645.0  8.183683e+05
 76   1989.0